In [36]:
%matplotlib inline
import matplotlib.pyplot as plt
import csv
from textblob import TextBlob
import pandas as pd
import sklearn
import pickle
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import learning_curve
import warnings
warnings.filterwarnings('ignore')

dataset_path = "../input/dataset24/cleanedData.csv"
df = pd.read_csv(dataset_path)

def mask(df, key, value):
    return df[df[key] == value]
pd.DataFrame.mask = mask

def encode_target(df, target_column, dest_col_name):
    
    df_mod = df.copy()
    targets = df_mod[target_column].unique()
    map_to_int = {name: n for n, name in enumerate(targets)}
    df_mod[dest_col_name] = df_mod[target_column].replace(map_to_int)

    return (df_mod, targets)

In [37]:
df, targets = encode_target(df, "ONTASK", "intONTASK")
print("ONTASK", targets)
df, targets = encode_target(df, "SCHOOL", "intSCHOOL")
print("SCHOOL", targets)
df, targets = encode_target(df, "Class", "intClass")
print("Class", targets)
df, targets = encode_target(df, "Activity", "intActivity")
print("Activity", targets)

In [38]:
import graphviz

In [39]:
features = ["intSCHOOL", "intClass", "intActivity"]
features_train, features_test, ontask_train, ontask_test = \
    train_test_split(df[features], df['intONTASK'], train_size=0.7, test_size=0.3)

print (len(features_train), len(features_test), len(features_train) + len(features_test))

In [40]:
clf = LinearSVC(random_state=0)
clf.fit(features_train, ontask_train)

In [41]:
print(clf.coef_)
print(clf.intercept_)
print(clf.predict(features_test))

In [42]:
scores = cross_val_score(clf, features_train, ontask_train, scoring='neg_mean_squared_error', cv=10)
print("Folds: {0}, mean squared error: {1:.2} std: {1:.2}".format(len(scores),np.mean(np.abs(scores)),np.std(scores)))

In [43]:
print(scores)
1+scores.mean(), scores.std()

In [44]:
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=-1, train_sizes=np.linspace(.1, 1.0, 5)):
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt

In [45]:
%time plot_learning_curve(clf, "accuracy vs. training set size", features_train, ontask_train, cv=5)